In [1]:
import ee
import datetime
import os
import itertools
import sys
import collections

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

# append the parent path into sys-path so wen can import necessary modules
sys.path.append('../')

In [3]:
from BackGround_modules.Class_1_Make_fourier_imgs import Make_Fourier

#### Step_0_Define_Basic_Parameters

In [4]:
# imprt North_China_Plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [5]:
# Define the time-range
# just get one_range from every different sensors
year_range = [(f'{i}-01-01',f'{i+2}-12-31') for i in range(1990,2018,3)]

In [6]:
year_range

[('1990-01-01', '1992-12-31'),
 ('1993-01-01', '1995-12-31'),
 ('1996-01-01', '1998-12-31'),
 ('1999-01-01', '2001-12-31'),
 ('2002-01-01', '2004-12-31'),
 ('2005-01-01', '2007-12-31'),
 ('2008-01-01', '2010-12-31'),
 ('2011-01-01', '2013-12-31'),
 ('2014-01-01', '2016-12-31'),
 ('2017-01-01', '2019-12-31')]

#### Step_1_Loop through each year_range to create the harmonic imgs

In [9]:
Fourier_imgs = {}
Amplitude_Phase_imgs = {}

for start,end in year_range:
    
    # define the year_name
    year_name = f'{start[:4]}_{end[:4]}'
    
    # instatiate the instance
    Fourier_instance = Make_Fourier(start_date=start,end_date=end,
                                   area=North_China_Plain)
    
    # make result imgs
    Fourier_instance.Stp_1_Create_hamonic_names()
    Fourier_instance.Stp_2_Add_harmonics()
    Fourier_instance.Stp_3_Harmonic_fit()
    
    # Fourier img
    Fourier_imgs[year_name] = Fourier_instance.harmonicTrendCoefficients
    
    # amplitude_phase img
    Amplitude_Phase_imgs[year_name] = Fourier_instance.Amplitude_Phase_img

Analyzing the images of 1990_1992
Analyzing the images of 1993_1995
Analyzing the images of 1996_1998
Analyzing the images of 1999_2001
Analyzing the images of 2002_2004
Analyzing the images of 2005_2007
Analyzing the images of 2008_2010
Analyzing the images of 2011_2013
Analyzing the images of 2014_2016
Analyzing the images of 2017_2019


##### Step_2_Export the imgs

In [24]:
# here use the clip to export part of the reserach area, change to full area if was for whole area analysis
clip = ee.Geometry.MultiPolygon(
        [[[[116.39831522048985, 39.46687647601767],
           [116.39831522048985, 38.80005028254205],
           [117.23876932205235, 38.80005028254205],
           [117.23876932205235, 39.46687647601767]]],
         [[[113.62521238026795, 35.51176132219713],
           [113.62521238026795, 35.11731413601196],
           [114.14294309315858, 35.11731413601196],
           [114.14294309315858, 35.51176132219713]]],
         [[[120.34909146470943, 36.97156109000749],
           [120.34909146470943, 36.74080712198658],
           [120.6992806736938, 36.74080712198658],
           [120.6992806736938, 36.97156109000749]]],
         [[[116.84493735483619, 30.925303237941304],
           [116.84493735483619, 30.925303237941304],
           [116.84493735483619, 30.925303237941304],
           [116.84493735483619, 30.925303237941304]]],
         [[[116.79961875132057, 30.93708312491555],
           [116.79961875132057, 30.73308670996162],
           [117.07977011850807, 30.73308670996162],
           [117.07977011850807, 30.93708312491555]]]])

In [27]:
for year,img in Amplitude_Phase_imgs.items():

    # construct the export name    
    export_name = f'Amplitude_Phase_{year}'
    
    # here only export the year of 2008-2010, change to full years for fully analysis
    if year == '2008_2010':
        # construct the export task
        task = ee.batch.Export.image.toCloudStorage(image = img,
                                                    description=export_name,
                                                    bucket='north_china_plain',
                                                    fileNamePrefix=export_name,
                                                    region= clip,
                                                    scale=30,
                                                    maxPixels=int(1e13),
                                                    skipEmptyTiles=True)
        task.start()

        # print out the process
        print(export_name)

Amplitude_Phase_2008_2010
